## Pandas Exercise

### Part 1: Aggregation, Group By and Sort Review

Below is a dataframe, with values as a list-of-lists and columns as a list.

In [1]:
import pandas as pd

df = pd.DataFrame([[123,'xt23',20],
                   [123,'q45',2],
                   [123,'a89',25],
                   [77,'q45',3],
                   [77,'a89',30],
                   [92,'xt23',24],
                   [92,'m33',60],
                   [92,'a89',28]], columns=['userid','product','price'])
df

,userid,product,price
0,123,xt23,20
1,123,q45,2
2,123,a89,25
3,77,q45,3
4,77,a89,30
5,92,xt23,24
6,92,m33,60
7,92,a89,28


**Q: We want the maximum price anyone has paid.**

In [2]:
df.price.max()

60

**Q: We want the maximum price per user. Hint: Use groupby.**

In [3]:
df.groupby(['userid']).price.agg(['max']).reset_index()

,userid,max
0,77,30
1,92,60
2,123,25


**Q: What is the total amount paid by user?**

Note: Pandas will smartly leave out columns for which that aggregation doesn't have meaning.

In [4]:
df.groupby(['userid']).price.agg(['sum']).reset_index()

,userid,sum
0,77,33
1,92,112
2,123,47


**Q: Sort the dataframe by userid first, and then price, both ascending.**

In [5]:
df.sort_values(['userid', 'price'], ascending = [True, True])

,userid,product,price
3,77,q45,3
4,77,a89,30
5,92,xt23,24
7,92,a89,28
6,92,m33,60
1,123,q45,2
0,123,xt23,20
2,123,a89,25


### Part 2: Combining Techniques

**Q: We want the maximum price each user paid, and the product associated with that price.**

Note for SQL users: In SQL, you groupby and then sort, but in pandas, it's easier to do it the other way around.

In [6]:
#df.groupby(['userid'], as_index=False).agg({'price':'max'})
#idx=df.groupby(by='userid')['price'].idxmax()
#df.loc[idx]
idx=df.groupby(by='userid')['price'].idxmax()
df.loc[idx]

,userid,product,price
4,77,a89,30
6,92,m33,60
2,123,a89,25


### Part 3: Exploring The Index

Let's add a new column.

In [7]:
df['website'] = ['Amazon', 'Amazon', 'NewEgg', 'NewEgg',
                 'NewEgg', 'Amazon', 'Amazon', 'Amazon']
df

,userid,product,price,website
0,123,xt23,20,Amazon
1,123,q45,2,Amazon
2,123,a89,25,NewEgg
3,77,q45,3,NewEgg
4,77,a89,30,NewEgg
5,92,xt23,24,Amazon
6,92,m33,60,Amazon
7,92,a89,28,Amazon


**Q: What is the total amount paid by each user on each website?**

In [8]:
df.groupby(['userid', 'website']).agg({'price':'sum'})

price
userid website       
77     NewEgg      33
92     Amazon     112
123    Amazon      22
       NewEgg      25

Now let's do the same groupby as above, but set the as_index flag to "False". This will result in a flat table instead of the nested indexes.

In [9]:
df.groupby(['userid', 'website'], as_index=False).agg({'price':'sum'})

,userid,website,price
0,77,NewEgg,33
1,92,Amazon,112
2,123,Amazon,22
3,123,NewEgg,25


### Part 4: Merging

Let's now create a second table:

In [10]:
df2 = pd.DataFrame([
    [123, 'USA'], [77, 'Canada'], [92, 'USA']],
    columns=['userid', 'country'])
df2

,userid,country
0,123,USA
1,77,Canada
2,92,USA


We can combine the two tables using a merge function. What it does is, it will do a pairwise comparision of every row in table1 with every row in table2 and if the "on" condition matches, it will create a single row with columns from both those matched rows.

Merge of two tables with 5 rows each can give as little as 0 rows and as much as 25 rows.

    [1,2,3,4,5] merged with [6,7,8,9,10] will give 0 rows
    [1,2,3,4,5] merged with [1,2,3,4,5] will give 5 rows
    [1,1,1,1,1] merged with [1,1,1,1,1] will give 25 rows

In [11]:
pd.merge(df, df2, on='userid')

,userid,product,price,website,country
0,123,xt23,20,Amazon,USA
1,123,q45,2,Amazon,USA
2,123,a89,25,NewEgg,USA
3,77,q45,3,NewEgg,Canada
4,77,a89,30,NewEgg,Canada
5,92,xt23,24,Amazon,USA
6,92,m33,60,Amazon,USA
7,92,a89,28,Amazon,USA


**Q: What is the total amount paid per country?**

In [12]:
df = pd.merge(df, df2, on='userid')
df.groupby(['country'], as_index=False).agg({'price':'sum'})

,country,price
0,Canada,33
1,USA,159


**Q: What is the average amount paid per country and website?**

In [13]:
df.groupby(['country', 'website'], as_index=False).agg({'price':'sum'})

,country,website,price
0,Canada,NewEgg,33
1,USA,Amazon,134
2,USA,NewEgg,25


### Part 5: The Final Question - A Demo

Let's add another column: purchase date

In [14]:
df['date'] = ['2018-01-12',
              '2018-01-08',
              '2018-01-06',
              '2018-01-03',
              '2018-01-05',
              '2018-01-04',
              '2018-01-07',
              '2018-01-02']
df

,userid,product,price,website,country,date
0,123,xt23,20,Amazon,USA,2018-01-12
1,123,q45,2,Amazon,USA,2018-01-08
2,123,a89,25,NewEgg,USA,2018-01-06
3,77,q45,3,NewEgg,Canada,2018-01-03
4,77,a89,30,NewEgg,Canada,2018-01-05
5,92,xt23,24,Amazon,USA,2018-01-04
6,92,m33,60,Amazon,USA,2018-01-07
7,92,a89,28,Amazon,USA,2018-01-02


**Q: Here is a tricky task. For each row, I want the average purchase price for that user prior to that purchase.**

One option is to do some loops. But another solution is to just do a merge on itself and filter.

In [16]:
bg = df[['userid', 'price']]
#bg = bg[bg['userid'] == ]

In [17]:
test = pd.DataFrame({ 'Date' : ['2016-04-01','2016-04-01','2016-04-02',
                             '2016-04-02','2016-04-03','2016-04-04',
                             '2016-04-05','2016-04-06','2016-04-06'],
                      'User' : ['Mike','John','Mike','John','Mike','Mike',
                             'Mike','Mike','John'],
                      'Value' : [1,2,1,3,4.5,1,2,3,6]
                })

test

,Date,User,Value
0,2016-04-01,Mike,1.0
1,2016-04-01,John,2.0
2,2016-04-02,Mike,1.0
3,2016-04-02,John,3.0
4,2016-04-03,Mike,4.5
5,2016-04-04,Mike,1.0
6,2016-04-05,Mike,2.0
7,2016-04-06,Mike,3.0
8,2016-04-06,John,6.0


In [18]:
test['Date'] = pd.to_datetime(test['Date'])

df_test = test.groupby('User').apply(lambda x: x.set_index('Date').resample('1D').first())
#print(df_test)
df_test

User  Value
User Date                   
John 2016-04-01  John    2.0
     2016-04-02  John    3.0
     2016-04-03   NaN    NaN
     2016-04-04   NaN    NaN
     2016-04-05   NaN    NaN
     2016-04-06  John    6.0
Mike 2016-04-01  Mike    1.0
     2016-04-02  Mike    1.0
     2016-04-03  Mike    4.5
     2016-04-04  Mike    1.0
     2016-04-05  Mike    2.0
     2016-04-06  Mike    3.0

In [19]:
df1 = df_test.groupby(level=0)['Value'].apply(lambda x: x.shift().rolling(min_periods=1,window=2).mean()).reset_index(name='Value_Average_Past_2_days')       
df1

,User,Date,Value_Average_Past_2_days
0,John,2016-04-01,NaN
1,John,2016-04-02,2.00
2,John,2016-04-03,2.50
3,John,2016-04-04,3.00
4,John,2016-04-05,NaN
5,John,2016-04-06,NaN
6,Mike,2016-04-01,NaN
7,Mike,2016-04-02,1.00
8,Mike,2016-04-03,1.00
9,Mike,2016-04-04,2.75


### Movie / Wine Example

But first, quick question:

Say you merge (or 'join' if you come from SQL) two dataframes with 3 rows each, how many rows would you end up with?

Could be anything between 0-9.

Consider the following examples, where table x has users and the movies they like. And table y has users and the wines they line. And let's do a merge to come up with possible movie and wine pairings for any user. In case A, we get 0 rows, in case B, we get 3 rows and case C we get 9 rows.

#### Merge Two Tables with No IDs in Common

In [20]:
dfx = pd.DataFrame([[1,'Godfather'],[2,'Amelie'],[3,'Chicago']],columns=['userid','movies'])
dfx

,userid,movies
0,1,Godfather
1,2,Amelie
2,3,Chicago


In [21]:
dfy = pd.DataFrame([[4,'red'],[5,'white'],[6,'pink']],columns=['userid','wines'])
dfy

,userid,wines
0,4,red
1,5,white
2,6,pink


In [22]:
dfm = pd.merge(dfx,dfy,on='userid')
dfm

,userid,movies,wines


#### Merge Two Tables with IDs in Common (1 Value Per ID)

In [23]:
dfx = pd.DataFrame([[1,'Godfather'],[2,'Amelie'],[3,'Chicago']],columns=['userid','movies'])
dfx

,userid,movies
0,1,Godfather
1,2,Amelie
2,3,Chicago


In [24]:
dfy = pd.DataFrame([[1,'red'],[2,'white'],[3,'pink']],columns=['userid','wines'])
dfy

,userid,wines
0,1,red
1,2,white
2,3,pink


In [25]:
dfm = pd.merge(dfx,dfy,on='userid')
dfm

,userid,movies,wines
0,1,Godfather,red
1,2,Amelie,white
2,3,Chicago,pink


#### Merge Two Tables with IDs in Common (Multiple Values Per ID)

In [26]:
dfx = pd.DataFrame([[1,'Godfather'],[1,'Amelie'],[1,'Chicago']],columns=['userid','movies'])
dfx

,userid,movies
0,1,Godfather
1,1,Amelie
2,1,Chicago


In [27]:
dfy = pd.DataFrame([[1,'red'],[1,'white'],[1,'pink']],columns=['userid','wines'])
dfy

,userid,wines
0,1,red
1,1,white
2,1,pink


In [28]:
dfm = pd.merge(dfx,dfy,on='userid')
dfm

,userid,movies,wines
0,1,Godfather,red
1,1,Godfather,white
2,1,Godfather,pink
3,1,Amelie,red
4,1,Amelie,white
5,1,Amelie,pink
6,1,Chicago,red
7,1,Chicago,white
8,1,Chicago,pink


### Back to the Question

Now let's return to the original question: For each row, I want the average purchase price for that user prior to that purchase. Let's do a merge on itself and filter.

Here are the steps we're going to take:
1. **MERGE**: Join table on itself. For each userid / date combo, show me all userid / date / price combos.
1. **FILTER**: For each userid / date combo, keep only the userid / date / price combos that were from earlier. Filter everything else out.
1. **AGGREGATE**: For each userid / date combo, find the average price for the remaining rows.
1. **MERGE**: Combine these values with the original dataframe for the final result.
1. **SORT**: Sort to make the results look pretty.

#### MERGE: Join table on itself. For each userid / date combo, show me all userid / date / price combos.

In [29]:
df_date = df[['userid','date']]
df_date

,userid,date
0,123,2018-01-12
1,123,2018-01-08
2,123,2018-01-06
3,77,2018-01-03
4,77,2018-01-05
5,92,2018-01-04
6,92,2018-01-07
7,92,2018-01-02


In [30]:
df_all = df[['userid','price','date']]
df_all

,userid,price,date
0,123,20,2018-01-12
1,123,2,2018-01-08
2,123,25,2018-01-06
3,77,3,2018-01-03
4,77,30,2018-01-05
5,92,24,2018-01-04
6,92,60,2018-01-07
7,92,28,2018-01-02


In [31]:
df2 = pd.merge(df_date, df_all, on='userid')
df2

,userid,date_x,price,date_y
0,123,2018-01-12,20,2018-01-12
1,123,2018-01-12,2,2018-01-08
2,123,2018-01-12,25,2018-01-06
3,123,2018-01-08,20,2018-01-12
4,123,2018-01-08,2,2018-01-08
5,123,2018-01-08,25,2018-01-06
6,123,2018-01-06,20,2018-01-12
7,123,2018-01-06,2,2018-01-08
8,123,2018-01-06,25,2018-01-06
9,77,2018-01-03,3,2018-01-03


#### FILTER: For each userid / date combo, keep only the userid / date / price combos that were from earlier. Filter everything else out.

In [32]:
df3 = df2[df2['date_x'] > df2['date_y']]
df3

,userid,date_x,price,date_y
1,123,2018-01-12,2,2018-01-08
2,123,2018-01-12,25,2018-01-06
5,123,2018-01-08,25,2018-01-06
11,77,2018-01-05,3,2018-01-03
15,92,2018-01-04,28,2018-01-02
16,92,2018-01-07,24,2018-01-04
18,92,2018-01-07,28,2018-01-02


#### **AGGREGATE**: For each userid / date combo, find the average price for the remaining rows.

In [33]:
df4 = df3.groupby(['userid','date_x'])[['price']].mean()
df4.rename(columns={'price': 'avg_prior_price'}, inplace=True)
df4

avg_prior_price
userid date_x                     
77     2018-01-05              3.0
92     2018-01-04             28.0
       2018-01-07             26.0
123    2018-01-08             25.0
       2018-01-12             13.5

#### **MERGE**: Combine these values with the original dataframe for the final result.

In [34]:
df

,userid,product,price,website,country,date
0,123,xt23,20,Amazon,USA,2018-01-12
1,123,q45,2,Amazon,USA,2018-01-08
2,123,a89,25,NewEgg,USA,2018-01-06
3,77,q45,3,NewEgg,Canada,2018-01-03
4,77,a89,30,NewEgg,Canada,2018-01-05
5,92,xt23,24,Amazon,USA,2018-01-04
6,92,m33,60,Amazon,USA,2018-01-07
7,92,a89,28,Amazon,USA,2018-01-02


In [35]:
df4.index

MultiIndex(levels=[[77, 92, 123], ['2018-01-04', '2018-01-05', '2018-01-07', '2018-01-08', '2018-01-12']],
           labels=[[0, 1, 1, 2, 2], [1, 0, 2, 3, 4]],
           names=['userid', 'date_x'])

In [36]:
final = df.merge(df4,
                 left_on=['userid','date'],
                 right_on=['userid','date_x'],
                 right_index=True,
                 how='left')
final

,userid,product,price,website,country,date,avg_prior_price
0,123,xt23,20,Amazon,USA,2018-01-12,13.5
1,123,q45,2,Amazon,USA,2018-01-08,25.0
2,123,a89,25,NewEgg,USA,2018-01-06,NaN
3,77,q45,3,NewEgg,Canada,2018-01-03,NaN
4,77,a89,30,NewEgg,Canada,2018-01-05,3.0
5,92,xt23,24,Amazon,USA,2018-01-04,28.0
6,92,m33,60,Amazon,USA,2018-01-07,26.0
7,92,a89,28,Amazon,USA,2018-01-02,NaN


#### **SORT**: Sort to make the results look pretty.

In [37]:
final.sort_values(by=['userid','date'])

,userid,product,price,website,country,date,avg_prior_price
3,77,q45,3,NewEgg,Canada,2018-01-03,NaN
4,77,a89,30,NewEgg,Canada,2018-01-05,3.0
7,92,a89,28,Amazon,USA,2018-01-02,NaN
5,92,xt23,24,Amazon,USA,2018-01-04,28.0
6,92,m33,60,Amazon,USA,2018-01-07,26.0
2,123,a89,25,NewEgg,USA,2018-01-06,NaN
1,123,q45,2,Amazon,USA,2018-01-08,25.0
0,123,xt23,20,Amazon,USA,2018-01-12,13.5


# Pandas Assessment

In [39]:
# WARNING: If you close your browser, your work will be LOST!

# Q1: import numpy and pandas
import pandas as pd
import numpy as np

In [40]:
# Q2: Set options in pandas so it displays 30 `max_rows` 
#                                 no limit on `max_columns`
#                                 precision is 3


# various options in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.precision', 3)

In [41]:
# Q3: assign the versions of pandas and numpy to variables below
pd_version = pd.__version__
np_version = np.__version__

print("Pandas version:", pd_version)
print("Numpy version:", np_version)

Pandas version: 0.23.4
Numpy version: 1.15.4


In [42]:
# Q4: Read in the csv file "olive.csv" with pandas. 
#     The filename is stored in the variable `olive_path`
#     Assign it to variable `df`
df = pd.read_csv('olive.csv')
#df = pd.read_csv('http://s3.amazonaws.com/istreet-assets/cqj0wXOp6sICdTRWc5X2cw/olive.csv.txt')
# The fields are:

# Unnamed: 0
# region
# area
# palmitic
# palmitoleic
# stearic
# oleic
# linoleic
# linolenic
# arachidic
# eicosenoic
df

,Unnamed: 0,region,area,palmitic,palmitoleic,stearic,oleic,linoleic,linolenic,arachidic,eicosenoic
0,1.North-Apulia,1,1,1075,75,226,7823,672,36,60,29
1,2.North-Apulia,1,1,1088,73,224,7709,781,31,61,29
2,3.North-Apulia,1,1,911,54,246,8113,549,31,63,29
3,4.North-Apulia,1,1,966,57,240,7952,619,50,78,35
4,5.North-Apulia,1,1,1051,67,259,7771,672,50,80,46
5,6.North-Apulia,1,1,911,49,268,7924,678,51,70,44
6,7.North-Apulia,1,1,922,66,264,7990,618,49,56,29
7,8.North-Apulia,1,1,1100,61,235,7728,734,39,64,35
8,9.North-Apulia,1,1,1082,60,239,7745,709,46,83,33
9,10.North-Apulia,1,1,1037,55,213,7944,633,26,52,30


In [48]:
# Q5: How many rows and columns are there in `df`? Put your answer in a variable `df_shape`.
df_shape = df.shape
df_shape

(572, 11)

In [49]:
# Q6: assign the first 2 rows of `df` to `df_first_two`
df_first_two = df.iloc[0:2]
df_first_two

,Unnamed: 0,region,area,palmitic,palmitoleic,stearic,oleic,linoleic,linolenic,arachidic,eicosenoic
0,1.North-Apulia,1,1,1075,75,226,7823,672,36,60,29
1,2.North-Apulia,1,1,1088,73,224,7709,781,31,61,29


In [58]:
# Q7: How many distinct data types are there in `df`?
df_data_types_count =   len(np.unique(df.dtypes))# this should be an integer
#print(df.info())
#print(np.unique(df.dtypes))
df_data_types_count

2

In [59]:
# Q8: In `df`, create a new column 'sub_region_raw', which will be a copy of the column 'Unnamed: 0'
sub_region_raw = df[['Unnamed: 0']]
sub_region_raw

,Unnamed: 0
0,1.North-Apulia
1,2.North-Apulia
2,3.North-Apulia
3,4.North-Apulia
4,5.North-Apulia
5,6.North-Apulia
6,7.North-Apulia
7,8.North-Apulia
8,9.North-Apulia
9,10.North-Apulia


In [60]:
# Q9: In `df`, rename 'Unnamed: 0' to 'sub_region_desc'
df.rename(columns={'Unnamed: 0': 'sub_region_desc'}, inplace=True)
df.head(n=10)

,sub_region_desc,region,area,palmitic,palmitoleic,stearic,oleic,linoleic,linolenic,arachidic,eicosenoic
0,1.North-Apulia,1,1,1075,75,226,7823,672,36,60,29
1,2.North-Apulia,1,1,1088,73,224,7709,781,31,61,29
2,3.North-Apulia,1,1,911,54,246,8113,549,31,63,29
3,4.North-Apulia,1,1,966,57,240,7952,619,50,78,35
4,5.North-Apulia,1,1,1051,67,259,7771,672,50,80,46
5,6.North-Apulia,1,1,911,49,268,7924,678,51,70,44
6,7.North-Apulia,1,1,922,66,264,7990,618,49,56,29
7,8.North-Apulia,1,1,1100,61,235,7728,734,39,64,35
8,9.North-Apulia,1,1,1082,60,239,7745,709,46,83,33
9,10.North-Apulia,1,1,1037,55,213,7944,633,26,52,30


In [61]:
# Q10: In `df`, rename the column 'area' to: 'sub_region'
df.rename(columns={'area' : 'sub_region'}, inplace=True)
df.head(n=10)

,sub_region_desc,region,sub_region,palmitic,palmitoleic,stearic,oleic,linoleic,linolenic,arachidic,eicosenoic
0,1.North-Apulia,1,1,1075,75,226,7823,672,36,60,29
1,2.North-Apulia,1,1,1088,73,224,7709,781,31,61,29
2,3.North-Apulia,1,1,911,54,246,8113,549,31,63,29
3,4.North-Apulia,1,1,966,57,240,7952,619,50,78,35
4,5.North-Apulia,1,1,1051,67,259,7771,672,50,80,46
5,6.North-Apulia,1,1,911,49,268,7924,678,51,70,44
6,7.North-Apulia,1,1,922,66,264,7990,618,49,56,29
7,8.North-Apulia,1,1,1100,61,235,7728,734,39,64,35
8,9.North-Apulia,1,1,1082,60,239,7745,709,46,83,33
9,10.North-Apulia,1,1,1037,55,213,7944,633,26,52,30


In [63]:
# Q11: In `df`, get the unique values of the field 'region'
region_unique = df['region'].unique()
region_unique

array([1, 2, 3])

In [65]:
# Q12: In `df`, how many unique values of the field 'sub_region' are there?
sub_region_unique_count =   len(df['sub_region'].unique()) # should be an integer
sub_region_unique_count

9

In [78]:
# Q13: Lets take a look at the field `sub_region_desc`:

# >>> df['sub_region_desc'].head(5)
# 0    1.North-Apulia
# 1    2.North-Apulia
# 2    3.North-Apulia
# 3    4.North-Apulia
# 4    5.North-Apulia

#df['sub_region_desc'].str.split('.')
df['sub_region_desc'] = df['sub_region_desc'].apply(lambda x:  x.split('.')[-1])

# Looks like 'sub_region_desc' has line numbers attached to the beginning of region name. Remove those and get the unique values in that field, assign it to `srd_unique`
srd_unique = df['sub_region_desc'].unique()
srd_unique

array(['North-Apulia', 'Calabria', 'South-Apulia', 'Sicily',
       'Inland-Sardinia', 'Coast-Sardinia', 'Umbria', 'East-Liguria',
       'West-Liguria'], dtype=object)